# AI MAKERSPACE . COHORT 2 . DEMO DAY . PREPR

### IGNORE TO BE CLEANED UP LATER
This is the ...

### Step 1: Dependencies and Initialization

The basic setup and getting the OpenAI key to use their LLM and embeddings model.

In [26]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

^C
ERROR: Operation cancelled by user


In [ ]:
!pip install -qU qdrant-client

In [ ]:
!pip install -qU tiktoken pymupdf

In [ ]:
!pip install -qU ragas pandas

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

### Step 2: Get the Data - PDF file on the Internet

In [ ]:
#from langchain.document_loaders import PyMuPDFLoader

#docs = PyMuPDFLoader("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf").load()

from langchain.document_loaders import PyMuPDFLoader

# Adjust the URL to the direct download format
file_id = "1JeA-w4kvbI3GHk9Dh_j19_Q0JUDE7hse"
direct_url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Now load the document using the direct URL
docs = PyMuPDFLoader(direct_url).load()


### Step 3: Chunking the document
Chunking is breaking the document up into blocks of text that will be use by the retriever in our RAG pipeline.  We will be using the recursive character text splitter with a chunk size of 200, and no overlap.  Research for this type of app suggested a size of 500 and an overap of 50, but that did not fair well in answering specific questions.

In [ ]:
import tiktoken

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

### c) Step 4: Store chunks in a Qdrant vector storage using OpenAI embeddings
Qdrant, the vector database will be used.  It will be configured to use memory and will have a name of MetaFin.  LangChain has simple integration.

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="MetaFin",
)

### Step 5: Set up Retriever
Very easy to set up the retriever.

In [ ]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

### Step 6: Setup LangChain RAG pipeline using OpenAI 3.5 turbo LLM
We will be using Open AI 3.5 turbo as the LLM.  The prompt template is simple and provides basic guidance on what is expected as a financial advisor.  The heart of this RAG pipeline is the LangChain:
[query]->[retriever]->[prompt template]->[LLM]->[inference]
which can easily be integrated into a chatbot web application.


In [ ]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

Before proceeding to answer about which conference sessions the user should attend, be sure to ask them what key topics they are hoping to learn from the conference, and if there are any specific sessions they are keen on attending. Use the provided context to answer the user's query. You are a professional personal assistant for an executive professional in a high tech company. You help them plan for events and meetings.
You always review the provided event information. You can look up dates and location where event sessions take place from the document. If you do not know the answer, or cannot answer, please respond with "Insufficient data for further analysis, please try again". For each session you suggest, include bullet points with the session title, speaker, company, topic, AI industry relevance, details of their work in AI, main point likely to be made, and three questions to ask the speaker. You end your successful responses with "Is there anything else that I can help you with?". If the user says NO, or any other negative response, then you ask "How did I do?" >>
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

### Q & A Ready
OK. Lets try some questions.

In [ ]:
chainlit_question = "What session should I attend if I'm interested in Amazon Alexa?"
response = retrieval_augmented_qa_chain.invoke({"question" : chainlit_question })

In [ ]:
chainlin_answer = response["response"].content
print(chainlin_answer)

Based on the provided context, you should attend the following session related to Amazon Alexa:

- Session Title: [Virtual Exclusive] PRO TALK: Alexa on the Go: Building Location Aware Voice Experiences for Alexa
- Speaker: Craig Walls
- Company: VMware
- Topic: Building location-aware voice experiences for Amazon Alexa
- AI Industry Relevance: This session is relevant for voice application developers looking to create innovative voice experiences using Alexa.
- Details of Their Work in AI: Craig Walls is an engineer at VMware with expertise in developing applications for mobile Alexa devices.
- Main Point Likely to be Made: The session will focus on creating Alexa applications that can leverage location data to provide relevant information to users on-the-go.
- Questions to Ask the Speaker:
  1. How do you ensure user privacy and security when accessing location data for Alexa applications?
  2. What are the key challenges in developing voice experiences for mobile Alexa devices compa

In [ ]:
chainlit_question = "Are any of the speakers past or current employees of MongoDB?"
response = retrieval_augmented_qa_chain.invoke({"question" : chainlit_question })

In [ ]:
chainlin_answer = response["response"].content
print(chainlin_answer)

Yes, Dave Nielsen is a past and current employee of MongoDB. 

Is there anything else that I can help you with?


In [ ]:
for context in response["context"]:
  print(context)

page_content='Dave Nielsen \nHead of Community, MongoDB \nDave Nielsen is the Head of Community at MongoDB. Prior to that Dave was \nHead of Community at Harness, Traceable, Redis Labs, Strikeiron & PayPal. \nHe also co-founded CloudCamp \n \n \n \nWednesday February 21, 2024 5:00pm - 5:50pm PST  \nPRO Stage A  \n  AI & Machine Learning Conference  \n• \nCertificates AI & ML Certificate \n• \nTalk Type PRO WORKSHOP \n• \nTrack or Conference AI & Machine Learning Conference, Enterprise Summit \n• \nIn-Person/Virtual In Person \n5:00pm PST \nPRO WORKSHOP: Workflows of Highly Functional App & Data Engineering \nTeams  \nDavide Mauri, Microsoft, Principal Product Manager \nJerry Nixon, Microsoft, Engineer \n \nHow does a highly functional development team manage enterprise data \nengineering? They focus on the core principles of Schema Management, \nStandards Enforcement, Database Testing, and Automated Incremental \nDeployment. Microsoft brings an agonizing menu of choices to architects \

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the conference about?"})

In [ ]:
chainlin_answer = response["response"].content
print(chainlin_answer)

The conference is about driving software product design to new heights of innovation. It focuses on various key topics and best practices for software product design, including:

- Innovation and out-of-the-box thinking among software designers and developers.
- User-Centered Design strategies for product development.
- Agile and Lean Methodologies for streamlined development processes.
- Balancing creativity with practicality in product design.
- Cross-Functional Collaboration between designers, developers, and stakeholders.
- Embracing Emerging Technologies such as AI, blockchain, and IoT in product design.
- Ethical Considerations in software product design.
- Measuring Impact of innovative software product design on businesses and end-users.

Is there anything else that I can help you with?


In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "Who will be at the conference that I know?"})

In [ ]:
chainlin_answer = response["response"].content
print(chainlin_answer)

Based on the provided context, the following individuals will be at the conference that you may know:

- David Nicholas
  - Role: Software Consultant, Bitovi
  - Expertise: TypeScript and React
  - Industries: Blockchain, E-commerce, Medical Supply Manufacturing

- Rizèl Scarlett
  - Role: Developer Advocate at Block
  - Topic: The Beginner's Guide to Web5
  - Focus: Data sovereignty and privacy in the web

Is there anything else that I can help you with?


# BONUS - RAGAS 

In [7]:
!pip install -qU pymupdf

loader = PyMuPDFLoader(
    "/Users/SKTL/Desktop/Coding/VS-Code/AIE2-demo-day/DeveloperWeek2024_FullSchedule.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

NameError: name 'PyMuPDFLoader' is not defined

In [28]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/2202 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What was the outcome of the cancellation of mu...,"[employees across the FoA and RL segments, and...",The outcome of the cancellation of multiple da...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
1,What was the price per share of our Class A co...,[The trading price of our Class A common stock...,$38.00 per share,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
2,What is the amount of retained earnings as of ...,"[Stockholders' equity:\n \n \nCommon stock, $0...","82,070",simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
3,What is the responsibility of management in re...,[specified in the rules and forms of the Secur...,Our management is responsible for establishing...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
4,What limitations would the company face if the...,[seeks changes to our business. If we are unsu...,If the proposed order by the FTC is imposed in...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
5,What is an example of a situation where a chan...,"[period.\nFor purposes of this Policy, a Finan...",An example of a situation where a change in ac...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
6,What countries contributed to the majority of ...,[Accounts receivable are typically unsecured a...,"Western Europe, China, Brazil, Australia, Cana...",simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
7,What is the classification of assets based on ...,[Table of Contents\n \n \nFair Value Measureme...,The classification of assets based on unobserv...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
8,How does the adoption of new technologies affe...,[•\nusers adopt new technologies where our pro...,The adoption of new technologies can lead to t...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
9,What role do the bylaws play in potentially ma...,[Delaware law and provisions in our certificat...,"The bylaws could potentially make a merger, te...",simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True


In [29]:
testset.test_data[0]

DataRow(question='What was the outcome of the cancellation of multiple data center projects?', contexts=['employees across the FoA and RL segments, and a pivot towards a next generation data center design, including cancellation of multiple data center projects\n(the 2022 Restructuring). As of December\xa031, 2023, we have completed the data center initiatives and the 2022 employee layoffs, and substantially completed\nthe facilities consolidation initiatives.\n(1)\n104'], ground_truth='The outcome of the cancellation of multiple data center projects was the completion of the data center initiatives.', evolution_type='simple', metadata=[{'source': 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf', 'file_path': 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf', 'page': 104, 'total_pages': 147, 'format': 'PDF 1.4', 'title': '0001326801-24-000012', 'author': 'EDGAR® Online LLC, a subsidiary of OTC 

In [30]:
test_df = testset.to_pandas()

In [31]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What was the outcome of the cancellation of mu...,"[employees across the FoA and RL segments, and...",The outcome of the cancellation of multiple da...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
1,What was the price per share of our Class A co...,[The trading price of our Class A common stock...,$38.00 per share,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
2,What is the amount of retained earnings as of ...,"[Stockholders' equity:\n \n \nCommon stock, $0...","82,070",simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
3,What is the responsibility of management in re...,[specified in the rules and forms of the Secur...,Our management is responsible for establishing...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
4,What limitations would the company face if the...,[seeks changes to our business. If we are unsu...,If the proposed order by the FTC is imposed in...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
5,What is an example of a situation where a chan...,"[period.\nFor purposes of this Policy, a Finan...",An example of a situation where a change in ac...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
6,What countries contributed to the majority of ...,[Accounts receivable are typically unsecured a...,"Western Europe, China, Brazil, Australia, Cana...",simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
7,What is the classification of assets based on ...,[Table of Contents\n \n \nFair Value Measureme...,The classification of assets based on unobserv...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
8,How does the adoption of new technologies affe...,[•\nusers adopt new technologies where our pro...,The adoption of new technologies can lead to t...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
9,What role do the bylaws play in potentially ma...,[Delaware law and provisions in our certificat...,"The bylaws could potentially make a merger, te...",simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True


In [32]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [33]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [34]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [35]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [36]:
response_dataset[0]

{'question': 'What was the outcome of the cancellation of multiple data center projects?',
 'answer': 'The outcome of the cancellation of multiple data center projects was not explicitly mentioned in the provided context. \n\nIs there anything else that I can help you with?',
 'contexts': ["• \nIn-Person/Virtual Virtual \n3:00pm PST \n[Virtual Exclusive] Mastering the Multifaceted Challenges of Multi-Cloud SaaS \nOperations  \nShikha Srivsatava, IBM, Distinguished Engineer and Master Inventor \n \nSaaS vendors today are challenged to make their SaaS available in multiple \nclouds to meet the customers where they are. Running a SaaS (Software as a \nService) application across multiple cloud providers can offer benefits like \nredundancy, flexibility, and cost optimization, but it also presents several \nchallenges. Here are some of the key challenges SaaS vendors face when \noperating in multiple cloud environments:- Complexity and Heterogeneity: \nEach cloud provider has its own uniqu

In [37]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [38]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


In [ ]:
results

{'faithfulness': 0.7500, 'answer_relevancy': 0.3637, 'context_recall': 0.2632, 'context_precision': 0.0746, 'answer_correctness': 0.2990}

In [39]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What was the outcome of the cancellation of mu...,The outcome of the cancellation of multiple da...,[• \nIn-Person/Virtual Virtual \n3:00pm PST \n...,The outcome of the cancellation of multiple da...,NaN,0.000000,0.000000,0.0,0.215985
1,What was the price per share of our Class A co...,"Insufficient data for further analysis, please...","[Thursday February 22, 2024 9:30am - 9:55am PS...",$38.00 per share,NaN,0.725214,0.000000,0.0,0.181676
2,What is the amount of retained earnings as of ...,"Insufficient data for further analysis, please...","[Thursday February 22, 2024 9:30am - 9:55am PS...","82,070",0.000000,0.753913,0.000000,0.0,0.186236
3,What is the responsibility of management in re...,"In the context provided, the responsibility of...","[into ethical A/B testing practices, including...",Our management is responsible for establishing...,1.000000,0.962858,1.000000,0.0,0.713485
4,What limitations would the company face if the...,"Based on the provided context, the document do...",[There are 3 learning outcomes: \n-Identify wh...,If the proposed order by the FTC is imposed in...,NaN,0.000000,1.000000,0.0,0.219077
5,What is an example of a situation where a chan...,A change in accounting principles would not be...,[3:30pm PST \n[Virtual] PRO TALK: Restate: Dur...,An example of a situation where a change in ac...,1.000000,0.761663,0.000000,0.0,0.830095
6,What countries contributed to the majority of ...,"Insufficient data for further analysis, please...","[Waleed Kadous, Anyscale, Chief Scientist \n \...","Western Europe, China, Brazil, Australia, Cana...",0.000000,0.000000,0.000000,0.0,0.180934
7,What is the classification of assets based on ...,The fair value measurement at the reporting da...,[metrics and how to align them with organizati...,The classification of assets based on unobserv...,1.000000,0.702955,0.000000,0.0,0.661322
8,How does the adoption of new technologies affe...,"Based on the provided context, the adoption of...","[Mike Hansen, Sonatype, SVP of Engineering \n ...",The adoption of new technologies can lead to t...,1.000000,0.933377,0.000000,0.0,0.669317
9,What role do the bylaws play in potentially ma...,"In the context provided, there is no specific ...",[are the only way to ensure proper decentraliz...,"The bylaws could potentially make a merger, te...",1.000000,0.718414,1.000000,0.0,0.206138


### BASELINE RAGAS SCORE
WOW! Pretty impressive score:

faithfulness: 0.9444

answer_relevancy: 0.9466

context_recall: 0.9444

context_precision: 0.9244

answer_correctness: 0.6659

The only are for improvement would be in the "answer correctness"


### Changing the Retreiver
Changing the retreiver may help.  Looks like there are many from LangChain and 3rd party, or even roll my own.
It would be cool to leverage the ideas from the paper: "Dense Passage Retrieval for Open-Domain Question Answering" and build a new retriever, but for now, I'll settle on using Multi-Query Retriever from LangChain.

In [41]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=qdrant_retriever, llm=openai_chat_model)

In [42]:
# Need to experiment on using this prompt, or the original one
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [43]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


In [44]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(openai_chat_model, retrieval_qa_prompt)

In [45]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [46]:
response = retrieval_chain.invoke({"input": "What is the conference about?"})
print(response["answer"])

The conference is about driving software product design to new heights of innovation. It focuses on topics such as user-centered design, agile and lean methodologies, balancing creativity with practicality, cross-functional collaboration, embracing emerging technologies, ethical considerations, and measuring impact. Industry leaders, innovators, and thought leaders come together to share insights, strategies, and best practices for revolutionary software product design.


OK, looks good, lets Eval with Ragas

In [47]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])



In [ ]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Time to Evaluate

In [ ]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/90 [00:00<?, ?it/s]

In [ ]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What were the outcomes of the 2022 Restructuri...,"As of December 31, 2023, the outcomes of the 2...","[(the 2022 Restructuring). As of December 31, ...",The outcomes of the 2022 Restructuring include...,0.50,0.901826,1.0,0.916667,0.989179
1,How could changes in tax laws and regulations ...,Changes in tax laws and regulations could mate...,[Table of Contents\nOur future income tax rate...,Changes in tax laws or tax rulings could mater...,1.00,0.916948,1.0,1.000000,0.484104
2,What is the authorized number of shares for Cl...,"The company is authorized to issue 5,000 milli...",[Table of Contents\nNote 13. Stockholders' Equ...,"5,000 million shares of Class A common stock a...",1.00,0.932821,1.0,0.916667,0.990243
3,What are the penalties for non-compliance with...,The European General Data Protection Regulatio...,[Table of Contents\nconsiderable uncertainty a...,The GDPR includes significant penalties for no...,0.50,0.000000,1.0,0.833333,0.612189
4,What are the potential limitations and consequ...,The proposed consent order could limit the abi...,"[the composition of our board of directors, an...",If the proposed consent order is imposed in it...,1.00,0.951031,1.0,1.000000,0.865745
5,What is considered a material misstatement acc...,"According to the given context, an error in th...",[(h) “Financial Restatement” means a restateme...,An error that would result in a material misst...,1.00,0.956192,1.0,0.950000,0.735095
6,Where did a majority of the revenue outside of...,A majority of the revenue outside of the Unite...,"[$\n50,150 \n$\n51,541 \nEurope \n31,210 \n26,...","Customers located in western Europe, China, Br...",0.50,1.000000,0.0,0.770833,0.586519
7,What is the significance of quoted prices in a...,Quoted prices in active markets play a signifi...,"[required to be recorded at fair value, we con...",The significance of quoted prices in active ma...,1.00,0.967192,1.0,1.000000,0.881835
8,How does the adoption of new technologies affe...,The adoption of new technologies increases our...,"[products, and standards that we do not contro...",The adoption of new technologies can lead to t...,1.00,0.942882,1.0,0.667857,0.719509
9,How does Delaware law and its anti-takeover pr...,Delaware law and its anti-takeover provisions ...,[controlled company could make our Class A com...,Delaware law and its anti-takeover provisions ...,1.00,0.902979,1.0,1.000000,0.675830


In [ ]:
advanced_retrieval_results

{'faithfulness': 0.9028, 'answer_relevancy': 0.8868, 'context_recall': 0.9444, 'context_precision': 0.9309, 'answer_correctness': 0.7515}

# W00T!
Our answer correctness has improved and so has context precision!

But at a cost to our faithfulness and answer relevancy.
More research required!


faithfulness: 0.9028 (- 0.416)

answer_relevancy: 0.8868 (-0.598)

context_recall: 0.9444 (No Change)

context_precision: 0.9309 (+0.065)

answer_correctness: 0.7515 (+0.856)
